In [5]:
from shapely.geometry import Point, mapping
from rasterio.features import geometry_mask
import geopandas as gpd
import rasterio
import json
import numpy
from collections.abc import MutableMapping
# підглядав у Коноплітського та Менчинського

In [6]:
field_centroids = gpd.read_file('../dataset2/field_centroids.geojson')

In [7]:
def extract_properties(point, data_folder='../dataset2/soil_data'):
    with rasterio.open(data_folder + '/clay.tif') as src_clay, rasterio.open(data_folder + '/sand.tif') as src_sand, rasterio.open(data_folder + '/density.tif') as src_density:
        transform = src_clay.transform
        clay_band = src_clay.read(1)
        sand_band = src_sand.read(1)
        density_band = src_density.read(1)
    
    col, row = ~transform * point.coords[0]
    clay_value = clay_band[int(row), int(col)]
    sand_value = sand_band[int(row), int(col)]
    density_value = density_band[int(row), int(col)]
    
    return {'clay': int(clay_value), 'sand': int(sand_value), 'density': int(density_value)}

def extract_moisture(point, file='../dataset2/soil_moisture.tif'):
    with rasterio.open(file) as src:
        transform = src.transform
        moisture_band = src.read(1)
    
    col, row = ~transform * point.coords[0]
    moisture_value = moisture_band[int(row), int(col)]
    return int(moisture_value)

def convert(data):
    return int(data) if isinstance(data, numpy.uint8) else data

In [14]:
dataset = []
for _, row in field_centroids.iterrows():
    field_id = int(row['id'])
    field_name = row['Name']
    center_point = row['geometry']
    center_lon, center_lat = center_point.x, center_point.y
    
    soil_properties = extract_properties(center_point)
    soil_moisture = extract_moisture(center_point)
    feature = {
        'type': 'Feature',
        'properties': {
            'id': field_id,
            'name': field_name,
            'soil': soil_properties,
            'soil_moisture': soil_moisture
        },
        'geometry': [center_lon, center_lat]
    }
    
    dataset.append(feature)

with open('../dataset2/dataset_output.json', 'w') as f:
    json.dump(dataset, f, indent=4)

print(json.dumps(dataset, indent=4))

[
    {
        "type": "Feature",
        "properties": {
            "id": 1,
            "name": "Pin4",
            "soil": {
                "clay": 21,
                "sand": 38,
                "density": 1149
            },
            "soil_moisture": 2785
        },
        "geometry": [
            30.184082372130998,
            49.995266264307425
        ]
    },
    {
        "type": "Feature",
        "properties": {
            "id": 2,
            "name": "Pin17",
            "soil": {
                "clay": 21,
                "sand": 37,
                "density": 1212
            },
            "soil_moisture": 2813
        },
        "geometry": [
            30.250268297428843,
            49.98931843919832
        ]
    },
    {
        "type": "Feature",
        "properties": {
            "id": 3,
            "name": "Shev7-8-9",
            "soil": {
                "clay": 22,
                "sand": 35,
                "density": 1184
            },
      